Ce notebook va être consacré au preprocessing du dataset et l'entrainement d'un premier modèle simple pour établir une baseline.

---

# Load

In [2]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# pandas display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

base_path = 'data'
file_name = 'season.csv'
type_dict ={
    'LapStartTime': np.float32,
    'LapNumber': np.uint8,
    'LapTime': np.float32,
    'DriverNumber': "category",
    'Team' : "category",
    'Compound': "category",
    'TyreLife': np.uint8,
    'TrackStatus': "category",
    'Stint': np.uint8,
    'DistanceToDriverAhead': np.float32,
    'DriverAhead': "category",
    'PitStatus': "category",
    'IsAccurate': np.bool8,
    'Track': "category",
    'NumberOfPitStops': np.uint8,
    'Position' : np.uint8,
    'GapToLeader' : np.float32,
    'IntervalToPositionAhead' : np.float32,
    'LapsToLeader' : np.uint8,
    'TotalLaps' : np.uint8,
    'AirTemp': np.float32,
    'Humidity': np.float32,
    'Pressure': np.float32,
    'TrackTemp': np.float32,
    'WindDirection': np.float32,
    'WindSpeed': np.float32
    }

years = range(2019, 2023)

In [3]:
data = pd.DataFrame()

for year in years:
    path = os.path.join(base_path, str(year), file_name)
    year_csv = pd.read_csv(
            path,
            dtype=type_dict,
    )
    data = pd.concat([
        data,
        year_csv
    ])

# Preprocessing

In [4]:
## Rainfall -------------------------------------------------------------------

def process_rainfall(df): # Removes races with rain
    rain = df.groupby(['Year', 'RoundNumber', 'DriverNumber'])['Compound'].transform(lambda x: x[x.str.contains('INTERMEDIATE|WET')].count())
    return df[rain == 0].reset_index(drop=True)

## Incomplete races -----------------------------------------------------------
def incomplete_races(df):
    return df.groupby(['Year', 'RoundNumber', 'DriverNumber']).filter(lambda x: x['LapNumber'].max() + 3 >= x['TotalLaps'].max()).reset_index(drop=True)
## TrackName ------------------------------------------------------------------

def process_track_name(df):
    df['Track'] = df['Track'].str.replace(' ', '_')
    return df
## TrackStatus ----------------------------------------------------------------

def trackStatus_to_binary(df):
    trackStatus = df['TrackStatus']
    status = pd.Series(
        np.zeros(6, dtype=np.bool8),
        index=['Green', 'Yellow', 'SC', 'Red', 'VSC', 'SC_ending']
    )
    if "1" in trackStatus:
        status['Green'] = True
    if "2" in trackStatus:
        status['Yellow'] = True
    if "4" in trackStatus:
        status['SC'] = True
    if "5" in trackStatus:
        status['Red'] = True
    if "6" in trackStatus:
        status['VSC'] = True
    if "7" in trackStatus:
        status['SC_ending'] = True
    return status

def process_trackStatus(df):
    trackStatuses = df.apply(trackStatus_to_binary, axis=1)
    return pd.concat([df.drop('TrackStatus', axis=1), trackStatuses], axis=1)

## Missing Data ----------------------------------------------------------------

def process_missing_values(df):
    # TODO fill the missing values better
    df.fillna({
        'DistanceToDriverAhead': -1,
        'GapToLeader': -1,
        'IntervalToPositionAhead': -1,
    }, inplace=True)

    # drop all rows with missing laptime
    df.dropna(subset=['LapTime'], inplace=True)
    return df[df['LapNumber'] > 1].reset_index(drop=True)

## Datatypes -------------------------------------------------------------------

def process_datatypes(df):
    # boolean
    df['Green'] = df['Green'].astype('bool')
    df['Yellow'] = df['Yellow'].astype('bool')
    df['SC'] = df['SC'].astype('bool')
    df['Red'] = df['Red'].astype('bool')
    df['VSC'] = df['VSC'].astype('bool')
    df['SC_ending'] = df['SC_ending'].astype('bool')
    df['IsAccurate'] = df['IsAccurate'].astype('bool')
    df['Rainfall'] = df['Rainfall'].astype('bool')
    # category
    df['DriverNumber'] = df['DriverNumber'].astype('category')
    df['Team'] = df['Team'].astype('category')
    #df['Compound'] = df['Compound'].astype('category')
    df['DriverAhead'] = df['DriverAhead'].astype('category')
    #df['Track'] = df['Track'].astype('category')
    # float
    df['LapStartTime'] = df['LapStartTime'].astype('float32')
    df['LapTime'] = df['LapTime'].astype('float32')
    df['DistanceToDriverAhead'] = df['DistanceToDriverAhead'].astype('float32')
    df['GapToLeader'] = df['GapToLeader'].astype('float32')
    df['IntervalToPositionAhead'] = df['IntervalToPositionAhead'].astype('float32')
    df['AirTemp'] = df['AirTemp'].astype('float32')
    df['Humidity'] = df['Humidity'].astype('float32')
    df['Pressure'] = df['Pressure'].astype('float32')
    df['TrackTemp'] = df['TrackTemp'].astype('float32')
    df['WindDirection'] = df['WindDirection'].astype('float32')
    df['WindSpeed'] = df['WindSpeed'].astype('float32')
    # int
    df['LapNumber'] = df['LapNumber'].astype('uint8')
    df['TyreLife'] = df['TyreLife'].astype('uint8')
    df['Stint'] = df['Stint'].astype('uint8')
    df['NumberOfPitStops'] = df['NumberOfPitStops'].astype('uint8')
    df['Position'] = df['Position'].astype('uint8')
    df['LapsToLeader'] = df['LapsToLeader'].astype('uint8')
    df['TotalLaps'] = df['TotalLaps'].astype('uint8')
    return df

## Add target ------------------------------------------------------------------

def process_target(df):
    df['is_pitting'] = df['PitStatus'] == 'InLap'
    df['is_pitting'] = df['is_pitting'].astype('bool')
    return df

## Remove features -------------------------------------------------------------

def process_remove_features(df):
    df.drop(['LapStartTime', 'DriverNumber', 'Team', 'DriverAhead', 
    'AirTemp', 'Humidity', 'Pressure', 'Rainfall', 'TrackTemp', 'WindDirection', 'WindSpeed',
    'PitStatus', 'IsAccurate', 'Year', 'RoundNumber'], axis=1, inplace=True)
    return df

## Feature encoding ------------------------------------------------------------

from sklearn.preprocessing import OneHotEncoder

def process_feature_encoding(df):
    categorical_features = ['Compound', 'Track']
    one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    one_hot_encoder.fit(df[categorical_features])
    one_hot_encoded = one_hot_encoder.transform(df[categorical_features])
    one_hot_encoded = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_features))
    print("One hot : ", one_hot_encoded.shape)
    print("Data : ", df.shape)
    df = df.join(one_hot_encoded)
    df.drop(categorical_features, axis=1, inplace=True)
    return df, one_hot_encoder

def process_feature_encoding_new(df, encoder):
    categorical_features = ['Compound', 'Track']
    one_hot_encoded = encoder.transform(df[categorical_features])
    one_hot_encoded = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_features))
    df = df.join(one_hot_encoded)
    df.drop(categorical_features, axis=1, inplace=True)
    return df

In [5]:
def preprocess_pre_split(df):
    df = df.copy()
    print('Preprocessing data...', df.shape)
    df = process_rainfall(df)
    print('Rainfall processed...', df.shape)
    df = incomplete_races(df)
    df = process_track_name(df)
    print('TrackName processed...', df.shape)
    df = process_missing_values(df)
    print('Missing values processed...', df.shape)
    return df

def preprocess_post_split(df):
    df = process_trackStatus(df)
    print('TrackStatus processed...', df.shape)
    df = process_datatypes(df)
    print('Preprocessing data...', df.shape)
    df = process_remove_features(df)
    print('Features removed...', df.shape)
    return df

def preprocess_post_split_train(df):
    df = df.copy()
    df = process_target(df)
    print('Target processed...', df.shape)
    df, encoder = process_feature_encoding(df)
    print('Features encoded...', df.shape)

    df = preprocess_post_split(df)
    
    return df, encoder

def preprocess_post_split_test(df, encoder):
    df = df.copy()
    df = process_target(df)
    print('Target processed...', df.shape)
    df = process_feature_encoding_new(df, encoder)
    print('Features encoded...', df.shape)

    df = preprocess_post_split(df)

    return df

In [6]:
def get_races_grouped(df):
    return df.groupby(['Year', 'RoundNumber', 'DriverNumber'])

def get_train_test_split(df, test_size, return_groups=False):
    groups = get_races_grouped(df).groups
    groups_keys = list(groups.keys())
    np.random.shuffle(groups_keys)
    test_groups = groups_keys[:int(len(groups_keys) * test_size)]
    train_groups = groups_keys[int(len(groups_keys) * test_size):]
    test = df[df.apply(lambda x: (x['Year'], x['RoundNumber'], x['DriverNumber']) in test_groups, axis=1)].reset_index(drop=True)
    train = df[df.apply(lambda x: (x['Year'], x['RoundNumber'], x['DriverNumber']) in train_groups, axis=1)].reset_index(drop=True)
    if return_groups:
        return train, test, train.groupby(['Year', 'RoundNumber', 'DriverNumber']).groups, test.groupby(['Year', 'RoundNumber', 'DriverNumber']).groups
    return train, test

In [7]:
df = data.copy()
df = preprocess_pre_split(df)

train_df, test_df, train_groups, test_groups = get_train_test_split(df.copy(), test_size=0.2, return_groups=True)

train_df, encoder = preprocess_post_split_train(train_df)
test_df = preprocess_post_split_test(test_df, encoder)

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

X_train = train_df.drop(['is_pitting'], axis=1)
y_train = train_df['is_pitting']

X_test = test_df.drop(['is_pitting'], axis=1)
y_test = test_df['is_pitting']

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Preprocessing data... (89139, 29)
Rainfall processed... (78346, 29)
TrackName processed... (73105, 29)
Missing values processed... (70689, 29)
Target processed... (56622, 30)
One hot :  (56622, 32)
Data :  (56622, 30)
Features encoded... (56622, 60)
TrackStatus processed... (56622, 65)
Preprocessing data... (56622, 65)
Features removed... (56622, 50)
Target processed... (14067, 30)
Features encoded... (14067, 60)
TrackStatus processed... (14067, 65)
Preprocessing data... (14067, 65)
Features removed... (14067, 50)
(56622, 49) (14067, 49) (56622,) (14067,)


# Modelisation

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
def specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp)

def balanced_accuracy(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    return (specificity + sensitivity) / 2

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [5, 20, 100, None],
    'class_weight': ['balanced_subsample', 'balanced', {0: 1, 1: 10}],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy']
}

model = RandomForestClassifier(random_state=0)

scorer = make_scorer(
    balanced_accuracy,
    greater_is_better=True,
    needs_proba=False
)

grid_search = GridSearchCV(
    model,
    param_grid,
    cv=4,
    scoring=scorer,
    verbose=10,
    n_jobs=-1
)

In [10]:
grid_search.fit(X_train.values, y_train)

Fitting 4 folds for each of 216 candidates, totalling 864 fits


GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'class_weight': ['balanced_subsample', 'balanced',
                                          {0: 1, 1: 10}],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 20, 100, None],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 500, 1000]},
             scoring=make_scorer(balanced_accuracy), verbose=10)

In [11]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7776109032840486
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'n_estimators': 100}


In [12]:
grid_search.best_estimator_.predict(X_test).shape
y_test.shape

c:\Users\nelso\miniconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


(14067,)

In [13]:
from sklearn.metrics import classification_report
ypred = grid_search.best_estimator_.predict(X_test.values)
print(confusion_matrix(y_test, ypred))
print(classification_report(y_test, ypred))

[[10263  3403]
 [   54   347]]
              precision    recall  f1-score   support

       False       0.99      0.75      0.86     13666
        True       0.09      0.87      0.17       401

    accuracy                           0.75     14067
   macro avg       0.54      0.81      0.51     14067
weighted avg       0.97      0.75      0.84     14067



In [14]:
model = grid_search.best_estimator_

In [15]:
# Save model
import pickle
models_dir = 'models/balanced_accuracy'
os.makedirs(models_dir, exist_ok=True)
pickle.dump(model, open(models_dir + '/model.pkl', 'wb'))
pickle.dump(encoder, open(models_dir + '/encoder.pkl', 'wb'))

In [16]:
ranks = grid_search.cv_results_['rank_test_score']
params = grid_search.cv_results_['params']
scores = grid_search.cv_results_['mean_test_score']

results = pd.DataFrame({
    'rank': ranks,
    'class_weight': [p['class_weight'] for p in params],
    'criterion': [p['criterion'] for p in params],
    'max_depth': [p['max_depth'] for p in params],
    'max_features': [p['max_features'] for p in params],
    'n_estimators': [p['n_estimators'] for p in params],
    'score': scores
})

results.set_index('rank', inplace=True)
results.dropna(subset=['score'], inplace=True)
results.sort_values(by='score', ascending=False, inplace=True)
pd.set_option('display.max_rows', None)
results

,class_weight,criterion,max_depth,max_features,n_estimators,score
rank,,,,,,
1,balanced,entropy,5.0,None,100,0.777611
2,balanced_subsample,entropy,5.0,None,100,0.777436
3,balanced,entropy,5.0,None,500,0.776522
4,balanced_subsample,entropy,5.0,None,500,0.776101
5,balanced_subsample,entropy,5.0,None,1000,0.776044
6,balanced,entropy,5.0,None,1000,0.775787
7,balanced_subsample,gini,5.0,None,500,0.771459
8,balanced,gini,5.0,None,500,0.771166
9,balanced,gini,5.0,None,1000,0.770911
